### Introduction

In this notebook, I processed the data scrapped from part 1, to facilitate subsequent analysis

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')
pd.set_option('display.max_colwidth', 9999)

### Read in Data

In [2]:
df = pd.read_csv('straitstimes_articles_raw.csv', parse_dates=['Published Date (Standardised)'])
df

Published Date Published Date (Standardised)  \
0          July 18, 2007 at 2:00 PM                    2007-07-18   
1        August 13, 2008 at 5:00 AM                    2008-08-13   
2          April 5, 2010 at 6:00 AM                    2010-04-05   
3         April 19, 2012 at 6:00 AM                    2012-04-19   
4         April 22, 2012 at 6:00 AM                    2012-04-22   
...                             ...                           ...   
31244  October 25, 2023 at 12:00 PM                    2023-10-25   
31245  October 25, 2023 at 12:35 PM                    2023-10-25   
31246   October 25, 2023 at 2:55 PM                    2023-10-25   
31247   October 25, 2023 at 3:45 PM                    2023-10-25   
31248   October 25, 2023 at 4:50 PM                    2023-10-25   

                                                                                                      Title  \
0                                             From the ST archives: Student, 19, missing for over two weeks   
1                                                                                  Silent star of Singapore   
2                                        A RUNNER’S DIARY\n\n        It's crazy only if you're not prepared   
3                                                                      Top 29 pupils head for the Big Spell   
4                                                         Why celebrate a spelling champ in the digital age   
...                                                                                                     ...   
31244                  Mandatory treatment for senior with mental condition who caused maid’s death in fire   
31245                            Former president Halimah Yacob awarded Singapore’s highest civilian honour   
31246  Over three years’ jail for man who cheated 68 victims of $383k in one of S’pore’s worst rental scams   
31247  Porsche, 2 Rolls-Royces among 4 cars seized from Bukit Timah bungalow in $2.8b money laundering case   
31248                      Construction manager’s death by plywood fragment ruled work-related misadventure   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [3]:
df.dtypes

Published Date                           object
Published Date (Standardised)    datetime64[ns]
Title                                    object
Article                                  object
Links                                    object
dtype: object

### Check and Drop Missing Values

In [4]:
df.isnull().mean()

Published Date                   0.00000
Published Date (Standardised)    0.00000
Title                            0.00000
Article                          0.04909
Links                            0.00000
dtype: float64

News with missing articles seems to typically be for subscribers only

In [5]:
df[df['Article'].isnull()]

,Published Date,Published Date (Standardised),Title,Article,Links
8,"June 26, 2012 at 3:20 PM",2012-06-26,City Harvest's Kong Hee and 4 others to be charged on Wednesday,NaN,https://www.straitstimes.com/singapore/city-harvests-kong-hee-and-4-others-to-be-charged-on-wednesday
378,"January 21, 2013 at 3:18 PM",2013-01-21,Singapore aims for total fertility rate of 1.4 or 1.5 from 1.28: DPM Teo,NaN,https://www.straitstimes.com/singapore/singapore-aims-for-total-fertility-rate-of-14-or-15-from-128-dpm-teo-0
382,"January 21, 2013 at 12:57 PM",2013-01-21,Bigger baby bonus and Medisave grants for parents,NaN,https://www.straitstimes.com/singapore/bigger-baby-bonus-and-medisave-grants-for-parents-0
383,"January 21, 2013 at 12:54 PM",2013-01-21,Government releases new measures to promote marriage and parenthood,NaN,https://www.straitstimes.com/singapore/government-releases-new-measures-to-promote-marriage-and-parenthood-0
385,"January 21, 2013 at 12:53 PM",2013-01-21,Young families moved to front of BTO queue,NaN,https://www.straitstimes.com/singapore/young-families-moved-to-front-of-bto-queue
...,...,...,...,...,...
29580,"November 18, 2021 at 3:56 PM",2021-11-18,Subscriber Picks: Do you know how much your colleagues are earning?,NaN,https://www.straitstimes.com/singapore/subscriber-picks-do-you-know-how-much-your-colleagues-are-earning
29588,"November 19, 2021 at 6:33 PM",2021-11-19,Evening Update: Today's headlines from The Straits Times on Nov 19,NaN,https://www.straitstimes.com/singapore/evening-update-todays-headlines-from-the-straits-times-on-nov-19-2021
29602,"November 19, 2021 at 7:00 PM",2021-11-19,BRANDED CONTENT\n\n Future-proofing organisations with a people-centric approach,NaN,https://www.straitstimes.com/singapore/jobs/future-proofing-organisations-with-a-people-centric-approach
29616,"November 20, 2021 at 8:22 AM",2021-11-20,Morning Briefing: Top stories from The Straits Times on Nov 20,NaN,https://www.straitstimes.com/singapore/morning-briefing-top-stories-from-the-straits-times-on-nov-20-2021


In [6]:
df = df.dropna(subset=['Article']).reset_index(drop=True)
df

Published Date Published Date (Standardised)  \
0          July 18, 2007 at 2:00 PM                    2007-07-18   
1        August 13, 2008 at 5:00 AM                    2008-08-13   
2          April 5, 2010 at 6:00 AM                    2010-04-05   
3         April 19, 2012 at 6:00 AM                    2012-04-19   
4         April 22, 2012 at 6:00 AM                    2012-04-22   
...                             ...                           ...   
29710  October 25, 2023 at 12:00 PM                    2023-10-25   
29711  October 25, 2023 at 12:35 PM                    2023-10-25   
29712   October 25, 2023 at 2:55 PM                    2023-10-25   
29713   October 25, 2023 at 3:45 PM                    2023-10-25   
29714   October 25, 2023 at 4:50 PM                    2023-10-25   

                                                                                                      Title  \
0                                             From the ST archives: Student, 19, missing for over two weeks   
1                                                                                  Silent star of Singapore   
2                                        A RUNNER’S DIARY\n\n        It's crazy only if you're not prepared   
3                                                                      Top 29 pupils head for the Big Spell   
4                                                         Why celebrate a spelling champ in the digital age   
...                                                                                                     ...   
29710                  Mandatory treatment for senior with mental condition who caused maid’s death in fire   
29711                            Former president Halimah Yacob awarded Singapore’s highest civilian honour   
29712  Over three years’ jail for man who cheated 68 victims of $383k in one of S’pore’s worst rental scams   
29713  Porsche, 2 Rolls-Royces among 4 cars seized from Bukit Timah bungalow in $2.8b money laundering case   
29714                      Construction manager’s death by plywood fragment ruled work-related misadventure   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [7]:
df.isnull().mean()

Published Date                   0.0
Published Date (Standardised)    0.0
Title                            0.0
Article                          0.0
Links                            0.0
dtype: float64

### Keep only news from 2013 onwards

Consideration: (1) News from more than 10 years ago may not be as meaningful; (2) from the value_counts(), there isn't a lot of articles from before 2013 anyway.

In [8]:
df['Published Year'] = df['Published Date (Standardised)'].dt.year
df['Published Year'].value_counts().sort_index()

2007       1
2008       1
2010       1
2012      33
2013    6788
2014    7677
2015    4287
2016     361
2017    1682
2018    1895
2019    1178
2020    1979
2021    2204
2022    1542
2023      86
Name: Published Year, dtype: int64

In [9]:
df = df[(df['Published Year']>=2013) & (df['Published Year'] <= 2023)].reset_index(drop=True)
df

Published Date Published Date (Standardised)  \
0        January 1, 2013 at 5:30 AM                    2013-01-01   
1       January 1, 2013 at 12:04 AM                    2013-01-01   
2       January 1, 2013 at 12:08 AM                    2013-01-01   
3        January 1, 2013 at 5:34 PM                    2013-01-01   
4        January 2, 2013 at 3:02 PM                    2013-01-02   
...                             ...                           ...   
29674  October 25, 2023 at 12:00 PM                    2023-10-25   
29675  October 25, 2023 at 12:35 PM                    2023-10-25   
29676   October 25, 2023 at 2:55 PM                    2023-10-25   
29677   October 25, 2023 at 3:45 PM                    2023-10-25   
29678   October 25, 2023 at 4:50 PM                    2023-10-25   

                                                                                                      Title  \
0                                         Changi Airport Group donates $360,000 to School Pocket Money Fund   
1                                    PICTURES\n\n        30,000 residents usher in the new year at Boon Lay   
2                                    PICTURES\n\n        Spectacular fireworks at Marina Bay ushers in 2013   
3                                                              13-year-old arrested for threats to bomb MBS   
4                                                    Man charged with eight counts of bribing SPH executive   
...                                                                                                     ...   
29674                  Mandatory treatment for senior with mental condition who caused maid’s death in fire   
29675                            Former president Halimah Yacob awarded Singapore’s highest civilian honour   
29676  Over three years’ jail for man who cheated 68 victims of $383k in one of S’pore’s worst rental scams   
29677  Porsche, 2 Rolls-Royces among 4 cars seized from Bukit Timah bungalow in $2.8b money laundering case   
29678                      Construction manager’s death by plywood fragment ruled work-related misadventure   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

### Extracting Article Categories

The categories of the article (e.g. court-crime, health) is sometimes within the HTML link. However, not all articles has them.

In [10]:
df['Category'] = df['Links'].apply(lambda x: x.split('/')[4] if len(x.split('/')) >= 6 else '')
df

Published Date Published Date (Standardised)  \
0        January 1, 2013 at 5:30 AM                    2013-01-01   
1       January 1, 2013 at 12:04 AM                    2013-01-01   
2       January 1, 2013 at 12:08 AM                    2013-01-01   
3        January 1, 2013 at 5:34 PM                    2013-01-01   
4        January 2, 2013 at 3:02 PM                    2013-01-02   
...                             ...                           ...   
29674  October 25, 2023 at 12:00 PM                    2023-10-25   
29675  October 25, 2023 at 12:35 PM                    2023-10-25   
29676   October 25, 2023 at 2:55 PM                    2023-10-25   
29677   October 25, 2023 at 3:45 PM                    2023-10-25   
29678   October 25, 2023 at 4:50 PM                    2023-10-25   

                                                                                                      Title  \
0                                         Changi Airport Group donates $360,000 to School Pocket Money Fund   
1                                    PICTURES\n\n        30,000 residents usher in the new year at Boon Lay   
2                                    PICTURES\n\n        Spectacular fireworks at Marina Bay ushers in 2013   
3                                                              13-year-old arrested for threats to bomb MBS   
4                                                    Man charged with eight counts of bribing SPH executive   
...                                                                                                     ...   
29674                  Mandatory treatment for senior with mental condition who caused maid’s death in fire   
29675                            Former president Halimah Yacob awarded Singapore’s highest civilian honour   
29676  Over three years’ jail for man who cheated 68 victims of $383k in one of S’pore’s worst rental scams   
29677  Porsche, 2 Rolls-Royces among 4 cars seized from Bukit Timah bungalow in $2.8b money laundering case   
29678                      Construction manager’s death by plywood fragment ruled work-related misadventure   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [11]:
df['Category'].value_counts()

                       20228
courts-crime            3732
transport               1393
health                  1174
education                793
environment              678
politics                 487
housing                  335
parenting-education      316
community                197
jobs                     145
consumer                 125
manpower                  76
Name: Category, dtype: int64

### Add HDB Town Involved and Filter

Consideration: There are several ways to categorise singapore locations

(1) By GRC / SMC - Not consistent manner as the boundaries changes every election \*cough\*

(2) By MRT stations - Too many MRT stations. Also, no clear boundary (where does Somerset ends and Orchard starts?), and not very meaningful (e.g. are we saying that the area around Somerset MRT station is less / more dangerous than Orchard MRT station?)

(3) By HDB towns - Most meaningful category. However, it means that non-HDB towns (e.g. Orchard with a lot of private housing) will be outside the scope of this analysis. Also, a lot of sub-towns (e.g. Chong Pang, Khatib, Boon Lay) are missed out (see: https://www.hdb.gov.sg/about-us/history/hdb-towns-your-home)

(4) URA Planning Areas - Very detailed, comes in both 'Planning Areas' and 'Subzone'. However, this is used for government planning and administrative purposes, and does not translate to much meaning on the ground. Some subzones such as 'Singapore General Hospital' (under Bukit Merah) or 'Port' (under Queenstown) doesn't really refer to an actual town (see: https://en.wikipedia.org/wiki/Planning_Areas_of_Singapore)

In the end, categorisation by HDB town is chosen

In [12]:
df['Article Lower'] = df['Article'].str.lower()
df

Published Date Published Date (Standardised)  \
0        January 1, 2013 at 5:30 AM                    2013-01-01   
1       January 1, 2013 at 12:04 AM                    2013-01-01   
2       January 1, 2013 at 12:08 AM                    2013-01-01   
3        January 1, 2013 at 5:34 PM                    2013-01-01   
4        January 2, 2013 at 3:02 PM                    2013-01-02   
...                             ...                           ...   
29674  October 25, 2023 at 12:00 PM                    2023-10-25   
29675  October 25, 2023 at 12:35 PM                    2023-10-25   
29676   October 25, 2023 at 2:55 PM                    2023-10-25   
29677   October 25, 2023 at 3:45 PM                    2023-10-25   
29678   October 25, 2023 at 4:50 PM                    2023-10-25   

                                                                                                      Title  \
0                                         Changi Airport Group donates $360,000 to School Pocket Money Fund   
1                                    PICTURES\n\n        30,000 residents usher in the new year at Boon Lay   
2                                    PICTURES\n\n        Spectacular fireworks at Marina Bay ushers in 2013   
3                                                              13-year-old arrested for threats to bomb MBS   
4                                                    Man charged with eight counts of bribing SPH executive   
...                                                                                                     ...   
29674                  Mandatory treatment for senior with mental condition who caused maid’s death in fire   
29675                            Former president Halimah Yacob awarded Singapore’s highest civilian honour   
29676  Over three years’ jail for man who cheated 68 victims of $383k in one of S’pore’s worst rental scams   
29677  Porsche, 2 Rolls-Royces among 4 cars seized from Bukit Timah bungalow in $2.8b money laundering case   
29678                      Construction manager’s death by plywood fragment ruled work-related misadventure   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [13]:
hdb_towns = ['sembawang', 'woodlands', 'yishun', 'ang mo kio', 'hougang', 'punggol', 'sengkang', 'serangoon',
            'bedok', 'pasir ris', 'tampines', 'bukit batok', 'bukit panjang', 'choa chu kang', 'clementi', 'jurong east',
            'jurong west', 'tengah', 'bishan' ,'bukit merah', 'bukit timah', 'central area', 'geylang', 'kallang', 'whampoa',
            'marine parade', 'queenstown', 'toa payoh']

In [14]:
df['Town Involved'] = df['Article Lower'].apply(lambda x: [town for town in hdb_towns if town in x])
df

Published Date Published Date (Standardised)  \
0        January 1, 2013 at 5:30 AM                    2013-01-01   
1       January 1, 2013 at 12:04 AM                    2013-01-01   
2       January 1, 2013 at 12:08 AM                    2013-01-01   
3        January 1, 2013 at 5:34 PM                    2013-01-01   
4        January 2, 2013 at 3:02 PM                    2013-01-02   
...                             ...                           ...   
29674  October 25, 2023 at 12:00 PM                    2023-10-25   
29675  October 25, 2023 at 12:35 PM                    2023-10-25   
29676   October 25, 2023 at 2:55 PM                    2023-10-25   
29677   October 25, 2023 at 3:45 PM                    2023-10-25   
29678   October 25, 2023 at 4:50 PM                    2023-10-25   

                                                                                                      Title  \
0                                         Changi Airport Group donates $360,000 to School Pocket Money Fund   
1                                    PICTURES\n\n        30,000 residents usher in the new year at Boon Lay   
2                                    PICTURES\n\n        Spectacular fireworks at Marina Bay ushers in 2013   
3                                                              13-year-old arrested for threats to bomb MBS   
4                                                    Man charged with eight counts of bribing SPH executive   
...                                                                                                     ...   
29674                  Mandatory treatment for senior with mental condition who caused maid’s death in fire   
29675                            Former president Halimah Yacob awarded Singapore’s highest civilian honour   
29676  Over three years’ jail for man who cheated 68 victims of $383k in one of S’pore’s worst rental scams   
29677  Porsche, 2 Rolls-Royces among 4 cars seized from Bukit Timah bungalow in $2.8b money laundering case   
29678                      Construction manager’s death by plywood fragment ruled work-related misadventure   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [15]:
df[df['Town Involved'].apply(lambda x: len(x) > 1)]

Published Date Published Date (Standardised)  \
16       January 2, 2013 at 11:04 AM                    2013-01-02   
22        January 3, 2013 at 8:23 PM                    2013-01-03   
55        January 4, 2013 at 1:41 PM                    2013-01-04   
71        January 6, 2013 at 4:20 PM                    2013-01-06   
75        January 7, 2013 at 9:33 PM                    2013-01-07   
...                              ...                           ...   
29657  September 13, 2023 at 4:20 PM                    2023-09-13   
29663     October 4, 2023 at 7:39 PM                    2023-10-04   
29666     October 4, 2023 at 6:33 PM                    2023-10-04   
29668     October 4, 2023 at 6:27 PM                    2023-10-04   
29671    October 20, 2023 at 4:00 AM                    2023-10-20   

                                                                                                                                   Title  \
16                                                                                          HDB resale prices up 2.5 per cent in Q4 2012   
22                                           PICTURES\n\n        Two new neighbourhood police centres in operation in Punggol, Woodlands   
55                                                                                          16-year-old arrested for series of break-ins   
71                                                                            Six more neighbourhoods to adopt community policing system   
75                                                                                      Snatch thief who targeted elderly women arrested   
...                                                                                                                                  ...   
29657                                            Jail for recalcitrant offender who whipped son with belt about 20 times and punched him   
29663                                                       Singapore to amend Free Trade Zones Act to combat illegal trading activities   
29666                                                 New laws to better safeguard against societies that may pose a threat in Singapore   
29668                                                            Evening Update: Today’s headlines from The Straits Times on Oct 4, 2023   
29671  BRANDED CONTENT\n\n        From relaxing strolls to birdwatching: How families like his are enjoying urban living close to nature   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

Consideration: 
- Remove rows with no HDB town mentioned because it does not contribute towards any analysis
- Remove rows with more than 1 town mentioned because it meant nothing town specific was covered in the news

In [16]:
df =df[df['Town Involved'].apply(lambda x: len(x) == 1)].reset_index(drop=True)
df

Published Date Published Date (Standardised)  \
0       January 2, 2013 at 7:49 PM                    2013-01-02   
1       January 2, 2013 at 5:10 PM                    2013-01-02   
2      January 2, 2013 at 11:09 AM                    2013-01-02   
3       January 3, 2013 at 5:39 PM                    2013-01-03   
4       January 3, 2013 at 5:37 PM                    2013-01-03   
...                            ...                           ...   
5464    October 4, 2023 at 7:16 PM                    2023-10-04   
5465    October 4, 2023 at 6:30 PM                    2023-10-04   
5466  October 25, 2023 at 11:05 AM                    2023-10-25   
5467   October 25, 2023 at 3:45 PM                    2023-10-25   
5468   October 25, 2023 at 4:50 PM                    2023-10-25   

                                                                                                     Title  \
0                                                     Teo Ho Pin: Contract with AIM benefits town councils   
1                                                               Man jailed for punching friend's boyfriend   
2                                                         Man jailed, fined and banned over fatal accident   
3                                                    Money mule jailed 4 1/2 years for laundering $855,000   
4                                          Two men found hiding in car boot arrested for illegal departure   
...                                                                                                    ...   
5464       Parliament debates on what constitutes basic needs after NTU report on adequate minimum incomes   
5465                        Workers’ Party’s Louis Chua seeks to enshrine flexi-work arrangements into law   
5466                                     $2.6b project to refresh Singapore’s oldest MRT lines almost done   
5467  Porsche, 2 Rolls-Royces among 4 cars seized from Bukit Timah bungalow in $2.8b money laundering case   
5468                      Construction manager’s death by plywood fragment ruled work-related misadventure   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [17]:
df['Town Involved'] = df['Town Involved'].apply(lambda x: x[0]).str.capitalize()
df

Published Date Published Date (Standardised)  \
0       January 2, 2013 at 7:49 PM                    2013-01-02   
1       January 2, 2013 at 5:10 PM                    2013-01-02   
2      January 2, 2013 at 11:09 AM                    2013-01-02   
3       January 3, 2013 at 5:39 PM                    2013-01-03   
4       January 3, 2013 at 5:37 PM                    2013-01-03   
...                            ...                           ...   
5464    October 4, 2023 at 7:16 PM                    2023-10-04   
5465    October 4, 2023 at 6:30 PM                    2023-10-04   
5466  October 25, 2023 at 11:05 AM                    2023-10-25   
5467   October 25, 2023 at 3:45 PM                    2023-10-25   
5468   October 25, 2023 at 4:50 PM                    2023-10-25   

                                                                                                     Title  \
0                                                     Teo Ho Pin: Contract with AIM benefits town councils   
1                                                               Man jailed for punching friend's boyfriend   
2                                                         Man jailed, fined and banned over fatal accident   
3                                                    Money mule jailed 4 1/2 years for laundering $855,000   
4                                          Two men found hiding in car boot arrested for illegal departure   
...                                                                                                    ...   
5464       Parliament debates on what constitutes basic needs after NTU report on adequate minimum incomes   
5465                        Workers’ Party’s Louis Chua seeks to enshrine flexi-work arrangements into law   
5466                                     $2.6b project to refresh Singapore’s oldest MRT lines almost done   
5467  Porsche, 2 Rolls-Royces among 4 cars seized from Bukit Timah bungalow in $2.8b money laundering case   
5468                      Construction manager’s death by plywood fragment ruled work-related misadventure   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

Remove articles where the title starts with 'branded content' or 'in partnership with', since these are sponsored content and not news

In [18]:
df = df[~df['Title'].str.lower().str.startswith(('branded content', 'in partnership with'))]
df

Published Date Published Date (Standardised)  \
0       January 2, 2013 at 7:49 PM                    2013-01-02   
1       January 2, 2013 at 5:10 PM                    2013-01-02   
2      January 2, 2013 at 11:09 AM                    2013-01-02   
3       January 3, 2013 at 5:39 PM                    2013-01-03   
4       January 3, 2013 at 5:37 PM                    2013-01-03   
...                            ...                           ...   
5464    October 4, 2023 at 7:16 PM                    2023-10-04   
5465    October 4, 2023 at 6:30 PM                    2023-10-04   
5466  October 25, 2023 at 11:05 AM                    2023-10-25   
5467   October 25, 2023 at 3:45 PM                    2023-10-25   
5468   October 25, 2023 at 4:50 PM                    2023-10-25   

                                                                                                     Title  \
0                                                     Teo Ho Pin: Contract with AIM benefits town councils   
1                                                               Man jailed for punching friend's boyfriend   
2                                                         Man jailed, fined and banned over fatal accident   
3                                                    Money mule jailed 4 1/2 years for laundering $855,000   
4                                          Two men found hiding in car boot arrested for illegal departure   
...                                                                                                    ...   
5464       Parliament debates on what constitutes basic needs after NTU report on adequate minimum incomes   
5465                        Workers’ Party’s Louis Chua seeks to enshrine flexi-work arrangements into law   
5466                                     $2.6b project to refresh Singapore’s oldest MRT lines almost done   
5467  Porsche, 2 Rolls-Royces among 4 cars seized from Bukit Timah bungalow in $2.8b money laundering case   
5468                      Construction manager’s death by plywood fragment ruled work-related misadventure   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [19]:
df = df.drop(['Article Lower'], axis=1).reset_index(drop=True)
df

Published Date Published Date (Standardised)  \
0       January 2, 2013 at 7:49 PM                    2013-01-02   
1       January 2, 2013 at 5:10 PM                    2013-01-02   
2      January 2, 2013 at 11:09 AM                    2013-01-02   
3       January 3, 2013 at 5:39 PM                    2013-01-03   
4       January 3, 2013 at 5:37 PM                    2013-01-03   
...                            ...                           ...   
5456    October 4, 2023 at 7:16 PM                    2023-10-04   
5457    October 4, 2023 at 6:30 PM                    2023-10-04   
5458  October 25, 2023 at 11:05 AM                    2023-10-25   
5459   October 25, 2023 at 3:45 PM                    2023-10-25   
5460   October 25, 2023 at 4:50 PM                    2023-10-25   

                                                                                                     Title  \
0                                                     Teo Ho Pin: Contract with AIM benefits town councils   
1                                                               Man jailed for punching friend's boyfriend   
2                                                         Man jailed, fined and banned over fatal accident   
3                                                    Money mule jailed 4 1/2 years for laundering $855,000   
4                                          Two men found hiding in car boot arrested for illegal departure   
...                                                                                                    ...   
5456       Parliament debates on what constitutes basic needs after NTU report on adequate minimum incomes   
5457                        Workers’ Party’s Louis Chua seeks to enshrine flexi-work arrangements into law   
5458                                     $2.6b project to refresh Singapore’s oldest MRT lines almost done   
5459  Porsche, 2 Rolls-Royces among 4 cars seized from Bukit Timah bungalow in $2.8b money laundering case   
5460                      Construction manager’s death by plywood fragment ruled work-related misadventure   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

### Process article to facilitate analysis

In [20]:
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [21]:
def lemmatize_text(lst):
    lemmatized_sentence = []
    for word, tag in lst:
        if tag is None:
            lemmatized_sentence.append(word)
        else:
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
            
    lemmatized_sentence = " ".join(lemmatized_sentence)
    return lemmatized_sentence

In [22]:
def remove_stopwords(text):
    token_list = tokenizer.tokenize(text)
    no_stopwords = [i for i in token_list if not i in stopwords.words('english')]
    return no_stopwords

In [23]:
df['Title Processed'] = df['Title'].str.lower()
df['Title POS'] = df['Title Processed'].apply(lambda x: nltk.pos_tag(nltk.word_tokenize(x)))
df['Title POS'] = df['Title POS'].apply(lambda x: list(map(lambda y: (y[0], pos_tagger(y[1])), x)))
df['Title Processed'] = df['Title POS'].apply(lambda x: lemmatize_text(x))
df['Title Processed'] = df['Title Processed'].apply(lambda x: remove_stopwords(x))
df = df.drop(['Title POS'], axis=1)
df

Published Date Published Date (Standardised)  \
0       January 2, 2013 at 7:49 PM                    2013-01-02   
1       January 2, 2013 at 5:10 PM                    2013-01-02   
2      January 2, 2013 at 11:09 AM                    2013-01-02   
3       January 3, 2013 at 5:39 PM                    2013-01-03   
4       January 3, 2013 at 5:37 PM                    2013-01-03   
...                            ...                           ...   
5456    October 4, 2023 at 7:16 PM                    2023-10-04   
5457    October 4, 2023 at 6:30 PM                    2023-10-04   
5458  October 25, 2023 at 11:05 AM                    2023-10-25   
5459   October 25, 2023 at 3:45 PM                    2023-10-25   
5460   October 25, 2023 at 4:50 PM                    2023-10-25   

                                                                                                     Title  \
0                                                     Teo Ho Pin: Contract with AIM benefits town councils   
1                                                               Man jailed for punching friend's boyfriend   
2                                                         Man jailed, fined and banned over fatal accident   
3                                                    Money mule jailed 4 1/2 years for laundering $855,000   
4                                          Two men found hiding in car boot arrested for illegal departure   
...                                                                                                    ...   
5456       Parliament debates on what constitutes basic needs after NTU report on adequate minimum incomes   
5457                        Workers’ Party’s Louis Chua seeks to enshrine flexi-work arrangements into law   
5458                                     $2.6b project to refresh Singapore’s oldest MRT lines almost done   
5459  Porsche, 2 Rolls-Royces among 4 cars seized from Bukit Timah bungalow in $2.8b money laundering case   
5460                      Construction manager’s death by plywood fragment ruled work-related misadventure   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [24]:
df['Article Processed'] = df['Article'].str.lower()
df['Article POS'] = df['Article Processed'].apply(lambda x: nltk.pos_tag(nltk.word_tokenize(x)))
df['Article POS'] = df['Article POS'].apply(lambda x: list(map(lambda y: (y[0], pos_tagger(y[1])), x)))
df['Article Processed'] = df['Article POS'].apply(lambda x: lemmatize_text(x))
df['Article Processed'] = df['Article Processed'].apply(lambda x: remove_stopwords(x))
df = df.drop(['Article POS'], axis=1)
df

Published Date Published Date (Standardised)  \
0       January 2, 2013 at 7:49 PM                    2013-01-02   
1       January 2, 2013 at 5:10 PM                    2013-01-02   
2      January 2, 2013 at 11:09 AM                    2013-01-02   
3       January 3, 2013 at 5:39 PM                    2013-01-03   
4       January 3, 2013 at 5:37 PM                    2013-01-03   
...                            ...                           ...   
5456    October 4, 2023 at 7:16 PM                    2023-10-04   
5457    October 4, 2023 at 6:30 PM                    2023-10-04   
5458  October 25, 2023 at 11:05 AM                    2023-10-25   
5459   October 25, 2023 at 3:45 PM                    2023-10-25   
5460   October 25, 2023 at 4:50 PM                    2023-10-25   

                                                                                                     Title  \
0                                                     Teo Ho Pin: Contract with AIM benefits town councils   
1                                                               Man jailed for punching friend's boyfriend   
2                                                         Man jailed, fined and banned over fatal accident   
3                                                    Money mule jailed 4 1/2 years for laundering $855,000   
4                                          Two men found hiding in car boot arrested for illegal departure   
...                                                                                                    ...   
5456       Parliament debates on what constitutes basic needs after NTU report on adequate minimum incomes   
5457                        Workers’ Party’s Louis Chua seeks to enshrine flexi-work arrangements into law   
5458                                     $2.6b project to refresh Singapore’s oldest MRT lines almost done   
5459  Porsche, 2 Rolls-Royces among 4 cars seized from Bukit Timah bungalow in $2.8b money laundering case   
5460                      Construction manager’s death by plywood fragment ruled work-related misadventure   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [25]:
df['Published Year'].value_counts().sort_index()

2013    1267
2014    1395
2015     751
2016      67
2017     392
2018     417
2019     244
2020     291
2021     376
2022     239
2023      22
Name: Published Year, dtype: int64

In [26]:
# df.to_csv('straitstimes_articles_processed.csv', index=False)